## Decision Model Testing 
Preliminaries for .toml files 

In [1]:
cd("../TestDrive")

In [2]:
] activate .

In [3]:
#] add JuMP#master

In [4]:
using JuMP, Ipopt, GLPK

In [5]:
using PyPlot
#using Plots
#gr()
; 

In [6]:
include("src/TestDrive.jl")

population_model_network (generic function with 1 method)

### Create scenario
To invoke the structs and data used in the popgen model for a single node

In [7]:
# REMOVE MIGRATION FROM NODES FOR FUTURE USE - 
# SHOULD ONLY OCCUR IN NETWORK STRUCT: 
migrate1 = rand((6*(4+8+6+1+6)), 2, 2) 

# User-named node  
firstnode = :FirstNode

# User-constructed node: (name, gene_data, stages, location, migration_matrix)
node = Node(:FirstNode, genetics_mcr, stages_aedes, temp_none, (37.87, 122.27), migrate1)

;

Go as far as initiating the u0: this can be done in the same way for both decision and popgen models

In [8]:
# init_node!(desired_node, gene_index)
u0 = init_node!(node, 4)
;

Extract necessary information from scenario 

In [9]:
# Network elements 
# node_count = length(get_nodes(network))

# Genetics elements 
genetics = get_genetics(node)
gN = length(genetics.all_genotypes)
cube = genetics.cube
S = genetics.S
Τ = genetics.Τ
Β = genetics.Β
Φ = genetics.Φ
Η = genetics.Η

;

In [10]:
# Lifestage elements 
(e, l, p, m, f) = get_allstages(node)

# Mortality 
μE = e.μ
μL = l.μ
μP = p.μ
μM = m.μ
μF = f.μ

# Substages 
nE = e.n
nL = l.n
nP = p.n
nM = m.n
nF = f.n

# Duration (only juveniles for now - though this should/might change with migration implemented)
qE = e.q
qL = l.q
qP = p.q

# Density 
densE = e.density
densL = l.density 
densP = p.density 
densM = m.density
densF = f.density 

;

In [11]:
# Controls (callbacks) -> define this later, for now just zero

controls = 0

;

### Define sets 
Where every set is a dimension, and not all the variables are necessarily subject to all the sets 

In [187]:
# Time
T = 1:10

# Stages and substages
SE = 1:nE
SL = 1:nL
SP = 1:nP
SM = 1:nM
SF = 1:nF*gN

# Genetics
G = 1:gN

# Nodes 
# N = 1:node_count

# Slicing u0 
E0 = u0[SE,:]
L0 = u0[nE+1:nL+nE,:]
P0 = u0[nE+nL+1:nE+nL+nP,:]
M0 = u0[nE+nL+nP+1,:]'
F0 = u0[nE+nL+nP+2:end,:];

### Define model and choose solver 
NB: Ipopt best for now

In [188]:
model = Model(with_optimizer(Ipopt.Optimizer));

### Define variables 
Each state or control that must be calculated using a decision variable

In [189]:
# Eggs
@variable(model, E[SE, G, T] >= 0)
set_start_value.(E[:,:,1].data, E0)

# Larvae 
@variable(model, L[SL, G, T] >= 0)
set_start_value.(L[:,:,1].data, L0)

# Pupae
@variable(model, P[SP, G, T] >= 0)
set_start_value.(P[:,:,1].data, P0)

# Males 
@variable(model, M[SM, G, T] >= 0)
set_start_value.(M[:,:,1].data, M0)

# Females
@variable(model, F[SF, G, T] >= 0)
set_start_value.(F[:,:,1].data, F0)
;

In [190]:
# View to check 
start_value.(P)
;

### Define constraints 
The dynamic equations of the model that employ a decision variable

EGGS

In [191]:
# Eggs
@constraint(model, E_con_A0[s = SE[1], g = G, t = T[1]],         
                   E[s, g, t] == E0[s, g] + 
                                 sum(cube[:,:,g].*Τ[:,:,g].*S[g]*Β[g].*F[:,:,t].data) - 
                                  E[s, g, t] * (μE*compute_density(densE, sum(E[:, :, t])) + qE*nE))
;

In [192]:
@constraint(model, E_con_A1[s = SE[1], g = G, t = T[2:end]],         
                   E[s, g, t] ==  E[s, g, t-1] +
                                  sum(cube[:,:,g].*Τ[:,:,g].*S[g]*Β[g].*F[:,:,t].data) - 
                                  E[s, g, t] * (μE*compute_density(densE, sum(E[:, :, t])) + qE*nE))

;

In [193]:
# Keep or no? 
@constraint(model, E_con_B0[s = SE[2:end], g = G, t = T[1]] ,    
                  E[s, g, t] ==   E0[s, g] + 
                                  qE*nE*E[s-1, g, t] - 
                                  E[s, g, t] * (μE*compute_density(densE, sum(E[:, :, t])) + qE*nE))
;

In [194]:
@constraint(model, E_con_B1[s = SE[2:end], g = G, t = T[2:end]] ,    
                  E[s, g, t] ==   E[s, g, t-1] +
                                  qE*nE*E[s-1, g, t] - 
                                  E[s, g, t] * (μE*compute_density(densE, sum(E[:, :, t])) + qE*nE))
;

LARVAE

In [195]:
# Larvae 
@constraint(model, L_con_A0[s = SL[1], g = G, t = T[1]],         
                   L[s, g, t] ==  L0[s, g] +
                                  qE * nE * E[end,g,t] -
                                  L[s, g, t] * (μL*compute_density(densL, sum(L[:, :, t])) + qL*nL))

;

In [196]:
@constraint(model, L_con_A1[s = SL[1], g = G, t = T[2:end]],         
                   L[s, g, t] ==  L[s, g, t-1] +
                                  qE * nE * E[end,g,t] -
                                  L[s, g, t] * (μL*compute_density(densL, sum(L[:, :, t])) + qL*nL))

;

In [197]:
@constraint(model, L_con_B0[s = SL[2:end], g = G, t = T[1]],  
                   L[s, g, t] == L0[s, g] +
                                 qL*nL*L[s-1, g, t] - 
                                 L[s, g, t] * (μL*compute_density(densL, sum(L[:, :, t])) + qL*nL))

;

In [198]:
@constraint(model, L_con_B1[s = SL[2:end], g = G, t = T[2:end]],  
                   L[s, g, t] == L[s, g, t-1] + 
                                 qL*nL*L[s-1, g, t] - 
                                 L[s, g, t] * (μL*compute_density(densL, sum(L[:, :, t])) + qL*nL))
;

PUPAE

In [199]:
# Pupae 
@constraint(model, P_con_A0[s = SP[1], g = G, t = T[1]],         
                   P[s, g, t] ==  P0[s, g] +
                                  qL * nL * L[end , g ,t] -
                                  P[s, g, t] * (μP*compute_density(densP, sum(L[:, :, t])) + qP*nP))

;

In [200]:
@constraint(model, P_con_A1[s = SP[1], g = G, t = T[2:end]],         
                   P[s, g, t] ==  P[s, g, t-1] + 
                                  qL * nL * L[end , g ,t] -
                                  P[s, g, t] * (μP*compute_density(densP, sum(L[:, :, t])) + qP*nP))

;

In [201]:
@constraint(model, P_con_B0[s = SP[2:end], g = G, t = T[1]] ,  
                   P[s, g, t] ==  P0[s, g] + 
                                  qP*nP*P[s-1, g, t] - 
                                  P[s, g, t] * (μP*compute_density(densP, sum(L[:, :, t])) + qP*nP))

;

In [202]:
@constraint(model, P_con_B1[s = SP[2:end], g = G, t = T[2:end]] ,  
                   P[s, g, t] ==  P[s, g, t-1] + 
                                  qP*nP*P[s-1, g, t] - 
                                  P[s, g, t] * (μP*compute_density(densP, sum(L[:, :, t])) + qP*nP))

;

MALES

In [203]:
# Males 

@constraint(model, M_con_0[s = SM, g = G, t = T[1]],         
                   M[s, g, t] ==  M0[s, g] +
                                  (1-Φ[g]) * qP * nP * P[end , g ,t] -
                                  μM * M[s, g, t] * compute_density(densM, sum(M[:, :, t]))) # + controls)

# TODO: Figure out how to implement controls here correctly
;

In [204]:
@constraint(model, M_con_1[s = SM, g = G, t = T[2:end]],         
                   M[s, g, t] ==  M[s, g, t-1] +
                                  (1-Φ[g]) * qP * nP * P[end , g ,t] -
                                  μM * M[s, g, t] * compute_density(densM, sum(M[:, :, t]))) # + controls)
;

In [205]:
M_con_1[:, :, :]
;

FEMALES

In [206]:
@NLconstraint(model, F_con_0[s = SF, g = G, t = T[1]], 
            F[s, g, t] == F0[s, g] + 
                        (M[1, g, t]*Η[g]/(1e-6 + sum(M[1, i, t]*Η[i] for i in G))*(Φ[g] * qP * nP * P[end, s, t]) -
                         μF * F[s, g, t])) # * compute_density(densF, sum(F[:, :, t])))

# Note difference necessary in S index definition between M terms and F terms 
# Re-include compute_density after reformulating so as not to use @NLconstraint
;

In [207]:
@NLconstraint(model, F_con_1[s = SF, g = G, t = T[2:end]], 
            F[s, g, t] == F[s, g, t-1] + 
                        (M[1, g, t]*Η[g]/(1e-6+sum(M[1, i, t]*Η[i] for i in G))*(Φ[g] * qP * nP * P[end, s, t]) -
                         μF * F[s, g, t])) # * compute_density(densF, sum(F[:, :, t])))

# See note above re: required difference in S index 
# See note above re: density in F
# NB:  eps() is unrecognized here, used 0.000001 instead
;

### Define objective function
In the case of one-shot optimization with no inequality constraints, can simply optimize!() without an objective function necessary.

In [208]:
result = optimize!(model)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    52670
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    33120

Total number of variables............................:     1500
                     variables with only lower bounds:     1500
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1500
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [209]:
value.(M)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:1
    Dimension 2, 1:6
    Dimension 3, 1:10
And data, a 1×6×10 Array{Float64,3}:
[:, :, 1] =
 3.35531e-13  3.35542e-13  3.21704e-13  500.404  1.25932e-13  3.11418e-13

[:, :, 2] =
 6.61729e-13  6.61843e-13  6.36811e-13  500.404  2.28406e-13  6.20602e-13

[:, :, 3] =
 9.53596e-13  9.54185e-13  9.19936e-13  500.404  3.04574e-13  8.98236e-13

[:, :, 4] =
 1.19724e-12  1.19934e-12  1.15807e-12  500.404  3.57065e-13  1.12801e-12

[:, :, 5] =
 1.38702e-12  1.39294e-12  1.34745e-12  500.404  3.90592e-13  1.30244e-12

[:, :, 6] =
 1.52474e-12  1.53881e-12  1.49271e-12  500.404  4.1156e-13  1.42154e-12

[:, :, 7] =
 1.61847e-12  1.64778e-12  1.60534e-12  500.404  4.2764e-13  1.49121e-12

[:, :, 8] =
 1.68093e-12  1.736e-12  1.70157e-12  500.404  4.47168e-13  1.52154e-12

[:, :, 9] =
 1.7274e-12  1.82255e-12  1.79971e-12  500.404  4.78365e-13  1.52502e-12

[:, :, 10] =
 1.77349e-12  1.92687e-12  1.91727e-12  500.404

In [211]:
value.(F)[:, 4, :]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, 1:10
And data, a 6×10 Array{Float64,2}:
   3.78003e-13    7.44777e-13    1.07918e-12  …    2.034e-12      2.06347e-12
   3.78021e-13    7.44909e-13    1.07979e-12       2.12918e-12    2.21687e-12
   3.2148e-13     6.25509e-13    9.30349e-13       2.01633e-12    2.12436e-12
 500.404        500.404        500.404           500.404        500.404      
   0.0            0.0            0.0               0.0            4.38129e-14
   2.64842e-13    4.21221e-13    6.98972e-13  …    1.65131e-12    1.63861e-12

In [ ]:
# SET THE OBJECTIVE FUNCTION FOR ONE-SHOT OPTIMIZATION

# how to correctly define? 
#@NLobjective(m, Min, sum(     )ˆ2 + 1e-6 * controls[t] for t = T[1:end]))


In [ ]:
# PRINT AND VIEW OBJECTIVE VALUE

#println("Objective value: ", getobjectivevalue(m))